In [11]:
from vpython import canvas, helix, rate, scene, sphere, vector

In [14]:
def animate_spring(t_max, spring_length=1.0):
    canvas()
    pivot = sphere(pos=vector(0.0, 0.0, 0.0), radius=0.1)
    mass = sphere(pos=vector(0.0, -spring_length, 0.0), radius=0.2)
    mass.v = vector(0.0, 0.0, 0.0)
    spring = helix(pos=pivot.pos - vector(0.0, pivot.radius, 0.0),
                   axis=mass.pos - pivot.pos + vector(0.0, mass.radius, 0.0),
                   radius=0.2, width=0.1)
    g = 9.81
    k = 10.0
    t = 0.0
    delta_t = 0.01
    while t <= t_max:
        rate(int(1.0/delta_t))
        mass.v += delta_t*vector(0.0, (-g - k*(spring_length - spring.length)), 0.0)
        mass.pos += delta_t*mass.v
        spring.axis = mass.pos - pivot.pos + vector(0.0, mass.radius, 0.0)
        t += delta_t

In [15]:
animate_spring(30.0)

<IPython.core.display.Javascript object>

KeyboardInterrupt: 